Follow this blog post
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [1]:
from gensim import corpora, models 
import numpy as np
import sys
import os
import gensim
import pickle
import nltk
from collections import Counter
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

python_root = './scripts'
sys.path.insert(0, python_root)

D:\data\chengyu\basics\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


#### Load data

In [2]:
paras = pickle.load(open('./data/processed_corpus.p', "rb")) 

In [3]:
print(paras[0])

['1', 'as', 'a', 'small', 'open', 'tourism', 'base', 'economy', 'st._lucia', 'be', 'highly_vulnerable_to_exogenous_shock', 'tourism', 'account', 'for', 'over', 'three_quarter', 'of', 'export', 'and', 'the', 'import_content', 'of', 'both', 'consumption', 'and', 'foreign_direct_investment', 'fdi', 'be', 'very', 'high', 'figure_1', 'the', 'economy', 'have', 'be', 'buffet', 'by', 'the', 'global', 'economic', 'downturn', 'which', 'have', 'hobble', 'the', 'tourism', 'and', 'construction', 'sector', 'with', 'potential', 'spillover', 'to', 'the', 'financial', 'sector']


#### Use Gensim for topic modeling

In [4]:
## remove stop words 
norm_tokenized_corpus = paras[:10000]
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.extend(['-PRON-','percent','’s','would','also',
                      'year','continue','include','give','may','new',
                      'however','well','help','since'])
norm_tokenized_corpus = [[token for token in p if token not in stopword_list] for p in norm_tokenized_corpus]

In [5]:
def _flatten(container):
    for i in container:
        if isinstance(i, (list,tuple)):
            for j in _flatten(i):
                yield j
        else:
            yield i

In [6]:
words = list(_flatten(norm_tokenized_corpus))
wc = Counter(words)
wc.most_common(30)

[('authority', 4601),
 ('fiscal', 4207),
 ('growth', 4181),
 ('bank', 4093),
 ('sector', 3601),
 ('policy', 3544),
 ('gdp', 3406),
 ('financial', 3275),
 ('increase', 3213),
 ('government', 3122),
 ('reform', 2966),
 ('public', 2860),
 ('debt', 2853),
 ('high', 2640),
 ('staff', 2619),
 ('tax', 2466),
 ('remain', 2390),
 ('rate', 2385),
 ('need', 2150),
 ('program', 2112),
 ('economic', 2074),
 ('market', 1995),
 ('risk', 1974),
 ('support', 1958),
 ('external', 1929),
 ('improve', 1914),
 ('low', 1818),
 ('revenue', 1721),
 ('investment', 1664),
 ('measure', 1638)]

In [7]:
# build dictionary
dictionary = corpora.Dictionary(norm_tokenized_corpus)
dictionary.filter_extremes(no_below=5,no_above=0.5, keep_n=10000)
# convert document into bow
corpus_bow = [dictionary.doc2bow(text) for text in norm_tokenized_corpus]
## comput tfidf feature vectors
tfidf = models.TfidfModel(corpus_bow)
corpus_tfidf = tfidf[corpus_bow]

In [8]:
len(dictionary)

6395

In [9]:
## a better way to print 
def print_topics_gensim(topic_model, total_topics=1,
                        weight_threshold=0.0001,
                        display_weights=False,
                        num_terms=None):
    
    for index in range(total_topics):
        topic = topic_model.show_topic(index,topn=num_terms)
        topic = [(word, round(wt,4)) 
                 for word, wt in topic 
                 if abs(wt) >= weight_threshold]
        if display_weights:
            print('Topic #'+str(index+1)+' with weights')
            print (topic[:num_terms] if num_terms else topic)
        else:
            print ('Topic #'+str(index+1)+' without weights')
            tw = [term for term, wt in topic]
            print (tw[:num_terms] if num_terms else tw)
        print

####  Latent Dirichlet Allocation

In [10]:
total_topics = 7

In [11]:
lda = models.LdaMulticore(corpus = corpus_bow,
                          id2word = dictionary,
                          iterations = 1000,
                          num_topics = total_topics)#,
                          #workers = 20) #alpha='auto',

In [12]:
print_topics_gensim(topic_model=lda,
                   total_topics = total_topics,
                   num_terms=20,
                   display_weights=True) 

Topic #1 with weights
[('fiscal', 0.0121), ('gdp', 0.0118), ('bank', 0.0105), ('authority', 0.0092), ('growth', 0.0074), ('policy', 0.0069), ('increase', 0.0066), ('sector', 0.0065), ('staff', 0.0065), ('financial', 0.0064), ('government', 0.0059), ('risk', 0.0051), ('rate', 0.0051), ('debt', 0.005), ('reform', 0.0048), ('market', 0.0047), ('support', 0.0046), ('deficit', 0.0046), ('need', 0.0043), ('public', 0.0042)]
Topic #2 with weights
[('authority', 0.0126), ('bank', 0.0105), ('government', 0.0103), ('debt', 0.0097), ('fiscal', 0.0087), ('public', 0.0083), ('sector', 0.0072), ('staff', 0.0065), ('exchange', 0.0058), ('gdp', 0.0058), ('rate', 0.0057), ('policy', 0.0056), ('reform', 0.0055), ('need', 0.0054), ('fund', 0.005), ('financial', 0.0049), ('program', 0.0046), ('growth', 0.0045), ('high', 0.0045), ('budget', 0.0043)]
Topic #3 with weights
[('growth', 0.0139), ('policy', 0.0092), ('increase', 0.0081), ('authority', 0.0071), ('financial', 0.0069), ('sector', 0.0068), ('high',

In [13]:
lda_model_filepath = 'data/lda_res'
lda.save(lda_model_filepath)

In [33]:
lda[corpus_bow[900]]

[(1, 0.952198)]

In [14]:
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

In [23]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus_bow, dictionary,n_jobs=2)

D:\data\chengyu\basics\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [24]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.027213  0.009060       1        1  18.621058
1      0.004395  0.000785       2        1  16.912355
0     -0.011228 -0.002968       3        1  16.486624
6     -0.015012  0.001077       4        1  16.108973
3     -0.006973 -0.006190       5        1  12.026642
4      0.023886 -0.028991       6        1  10.690646
5      0.032145  0.027228       7        1   9.153705, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
69    Default  2330.000000          tax  2330.000000  30.0000  30.0000
11    Default  3304.000000    financial  3304.000000  29.0000  29.0000
17    Default  3605.000000       sector  3605.000000  28.0000  28.0000
6     Default  2074.000000     economic  2074.000000  27.0000  27.0000
622   Default   664.000000        datum   664.000000  26.0000  26.0000
305   Default  1611.000000      project  1611.000000  25.0000  25.0000
129   Default  2805.000000         debt  2805.000000  24.0000  24.0000
137   Default  4082.000000       fiscal  4082.000000  23.0000  23.0000
585   Default  1132.000000   management  1132.000000  22.0000  22.0000
31    Default  4568.000000    authority  4568.000000  21.0000  21.0000
156   Default  3971.000000         bank  3971.000000  20.0000  20.0000
140   Default  3074.000000   government  3074.000000  19.0000  19.0000
253   Default  1338.000000       system  1338.000000  18.0000  18.0000
56    Default  1464.000000         fund  1464.000000  17.0000  17.0000
225   Default  1673.000000   investment  1673.000000  16.0000  16.0000
48    Default  1963.000000      support  1963.000000  15.0000  15.0000
97    Default  4137.000000       growth  4137.000000  14.0000  14.0000
174   Default  2339.000000       remain  2339.000000  13.0000  13.0000
231   Default   684.000000      million   684.000000  12.0000  12.0000
1999  Default   268.000000    statistic   268.000000  11.0000  11.0000
191   Default  1895.000000      improve  1895.000000  10.0000  10.0000
422   Default  2103.000000      program  2103.000000   9.0000   9.0000
95    Default  3314.000000          gdp  3314.000000   8.0000   8.0000
324   Default  1504.000000       budget  1504.000000   7.0000   7.0000
99    Default  3161.000000     increase  3161.000000   6.0000   6.0000
133   Default  1418.000000  expenditure  1418.000000   5.0000   5.0000
150   Default  2819.000000       public  2819.000000   4.0000   4.0000
646   Default  1474.000000     exchange  1474.000000   3.0000   3.0000
330   Default  1255.000000  development  1255.000000   2.0000   2.0000
64    Default  1672.000000      revenue  1672.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
422    Topic7   263.043243      program  2103.996826   0.3117  -5.1668
74     Topic7    87.996880         2006   446.975464   0.7658  -6.2619
136    Topic7   154.097305      finance  1006.510925   0.5144  -5.7016
337    Topic7    71.119507        grant   332.107544   0.8499  -6.4748
585    Topic7   164.761292   management  1132.778564   0.4631  -5.6347
174    Topic7   268.851349       remain  2339.956787   0.2273  -5.1450
527    Topic7   126.061867      enhance   784.366516   0.5629  -5.9024
31     Topic7   388.825928    authority  4568.324219  -0.0728  -4.7760
1024   Topic7   114.635574     director   722.277893   0.5504  -5.9974
56     Topic7   180.162430         fund  1464.079468   0.2959  -5.5453
489    Topic7   178.629059   strengthen  1460.952271   0.2895  -5.5539
140    Topic7   287.074310   government  3074.089111   0.0200  -5.0794
150    Topic7   261.732880       public  2819.293945   0.0141  -5.1718
330    Topic7   157.088943  development  1255.861450   0.3122  -5.6824
14     Topic7   244.008484         high  2600.186279   0.0249  -5.2420
597    Topic7   137.715927    framework  1034.587524   0.3744  

In [21]:
pyLDAvis.save_html(vis, './data/topic_lda.html')

In [25]:
pyLDAvis.display(vis)